# Test Indicators

In [37]:
pwd

'/Users/evawyf/1-Work/Trade/tws-native-bot'

In [38]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('iBot/data/HistoricalData_MES_FUT_1min_20241022.db')
# Read data from the database
query = "SELECT * FROM historical_data"
data = pd.read_sql_query(query, conn)
# Close the database connection
conn.close()

In [39]:
data

,timestamp,symbol,open,high,low,close,volume
0,20240923 06:30:00,MES,5774.50,5774.50,5770.00,5772.00,5359
1,20240923 06:31:00,MES,5772.00,5772.00,5768.50,5769.00,2951
2,20240923 06:32:00,MES,5768.75,5768.75,5766.50,5767.25,3400
3,20240923 06:33:00,MES,5767.25,5775.00,5766.50,5775.00,4372
4,20240923 06:34:00,MES,5774.75,5775.00,5769.75,5770.00,3450
...,...,...,...,...,...,...,...
19795,"b'\x00\x08""D\xdb\xca\x00\x18'",MES,5889.00,5889.25,5888.75,5889.00,b'g\x00\x00\x00\x00\x00\x00\x00'
19796,b'\x00`i<\xe9\xca\x00\x18',MES,5888.75,5889.00,5888.50,5888.75,b'B\x00\x00\x00\x00\x00\x00\x00'
19797,b'\x00\xb8\xb04\xf7\xca\x00\x18',MES,5889.00,5889.00,5887.75,5888.25,b'\xa6\x00\x00\x00\x00\x00\x00\x00'
19798,"b'\x00\x10\xf8,\x05\xcb\x00\x18'",MES,5888.25,5888.25,5887.75,5888.00,b'B\x00\x00\x00\x00\x00\x00\x00'


In [40]:
    # Convert timestamp to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')

    # Remove rows with invalid timestamp
    data = data.dropna(subset=['timestamp'])

    # Remove duplicate rows based on timestamp
    data = data.drop_duplicates(subset=['timestamp'])

    # Sort by timestamp
    data = data.sort_values('timestamp')

    # Set timestamp as index
    data.set_index('timestamp', inplace=True)

    # Print info about the cleaned dataset
    print(f"Dataset shape after cleaning: {data.shape}")
    print(f"Date range: from {data.index.min()} to {data.index.max()}")

Dataset shape after cleaning: (9900, 6)
Date range: from 2024-09-23 06:30:00 to 2024-10-22 13:59:00


In [41]:
data

,symbol,open,high,low,close,volume
timestamp,,,,,,
2024-09-23 06:30:00,MES,5774.50,5774.50,5770.00,5772.00,5359
2024-09-23 06:31:00,MES,5772.00,5772.00,5768.50,5769.00,2951
2024-09-23 06:32:00,MES,5768.75,5768.75,5766.50,5767.25,3400
2024-09-23 06:33:00,MES,5767.25,5775.00,5766.50,5775.00,4372
2024-09-23 06:34:00,MES,5774.75,5775.00,5769.75,5770.00,3450
...,...,...,...,...,...,...
2024-10-22 13:55:00,MES,5889.00,5889.25,5888.75,5889.00,103
2024-10-22 13:56:00,MES,5888.75,5889.00,5888.50,5888.75,66
2024-10-22 13:57:00,MES,5889.00,5889.00,5887.75,5888.25,166


### SuperTrend Basic

In [45]:
# basic version of SuperTrend 

def supertrend(data, period=10, multiplier=3):
    # Extract high, low, and close prices from the input data
    high = data['high']
    low = data['low']
    close = data['close']

    # Calculate True Range (TR)
    # TR is the greatest of the following:
    # 1. Current High - Current Low
    # 2. Absolute value of Current High - Previous Close
    # 3. Absolute value of Current Low - Previous Close
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join='inner').max(axis=1)
    
    # Calculate Average True Range (ATR)
    # ATR is an exponential moving average of TR
    atr = tr.ewm(com=period, min_periods=period).mean()

    # Calculate basic upper and lower bands
    # Basic bands are the midpoint of high and low, plus/minus a multiple of ATR
    hl2 = (high + low) / 2
    final_upperband = (hl2 + (multiplier * atr)).round(2)
    final_lowerband = (hl2 - (multiplier * atr)).round(2)

    # Initialize SuperTrend DataFrame
    supertrend = pd.DataFrame(index=data.index)
    supertrend['upperband'] = 0.00
    supertrend['lowerband'] = 0.00
    supertrend['supertrend'] = 0.00

    # Calculate SuperTrend
    for i in range(period, len(data)):
        # Adjust upper band
        if close.iloc[i] > final_upperband.iloc[i-1]:
            supertrend.iloc[i, supertrend.columns.get_loc('upperband')] = final_lowerband.iloc[i]
        else:
            supertrend.iloc[i, supertrend.columns.get_loc('upperband')] = final_upperband.iloc[i]
        
        # Adjust lower band
        if close.iloc[i] < final_lowerband.iloc[i-1]:
            supertrend.iloc[i, supertrend.columns.get_loc('lowerband')] = final_upperband.iloc[i]
        else:
            supertrend.iloc[i, supertrend.columns.get_loc('lowerband')] = final_lowerband.iloc[i]
        
        # Determine SuperTrend value
        if close.iloc[i] > supertrend.iloc[i-1, supertrend.columns.get_loc('upperband')]:
            supertrend.iloc[i, supertrend.columns.get_loc('supertrend')] = supertrend.iloc[i, supertrend.columns.get_loc('lowerband')]
        elif close.iloc[i] < supertrend.iloc[i-1, supertrend.columns.get_loc('lowerband')]:
            supertrend.iloc[i, supertrend.columns.get_loc('supertrend')] = supertrend.iloc[i, supertrend.columns.get_loc('upperband')]
        else:
            supertrend.iloc[i, supertrend.columns.get_loc('supertrend')] = supertrend.iloc[i-1, supertrend.columns.get_loc('supertrend')]

    return supertrend

# Calculate SuperTrend
st_data = supertrend(data)

# Merge SuperTrend data with original data
result = pd.concat([data, st_data], axis=1)

# Remove rows where SuperTrend info is 0
result = result[(result['upperband'] != 0) & (result['lowerband'] != 0) & (result['supertrend'] != 0)]

# Reset index if needed
result = result.reset_index()


In [46]:
# Calculate SuperTrend
st_data = supertrend(data)
# Merge SuperTrend data with original data
result = pd.concat([data, st_data], axis=1)

In [47]:
result

,symbol,open,high,low,close,volume,upperband,lowerband,supertrend
timestamp,,,,,,,,,
2024-09-23 06:30:00,MES,5774.50,5774.50,5770.00,5772.00,5359,0.00,0.00,0.00
2024-09-23 06:31:00,MES,5772.00,5772.00,5768.50,5769.00,2951,0.00,0.00,0.00
2024-09-23 06:32:00,MES,5768.75,5768.75,5766.50,5767.25,3400,0.00,0.00,0.00
2024-09-23 06:33:00,MES,5767.25,5775.00,5766.50,5775.00,4372,0.00,0.00,0.00
2024-09-23 06:34:00,MES,5774.75,5775.00,5769.75,5770.00,3450,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
2024-10-22 13:55:00,MES,5889.00,5889.25,5888.75,5889.00,103,5891.38,5886.62,5876.53
2024-10-22 13:56:00,MES,5888.75,5889.00,5888.50,5888.75,66,5891.05,5886.45,5876.53
2024-10-22 13:57:00,MES,5889.00,5889.00,5887.75,5888.25,166,5890.81,5885.94,5876.53


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_price_and_supertrend(data, num_bars=None):
    # Remove rows where SuperTrend is 0 and where market is closed
    plot_data = data[(data['supertrend'] != 0) & (data['upperband'] != 0) & (data['lowerband'] != 0)]
    plot_data = plot_data.dropna()  # Remove any NaN values which might represent market closed times

    # If num_bars is specified, take only the last num_bars
    if num_bars is not None:
        plot_data = plot_data.tail(num_bars)

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add candlestick trace
    fig.add_trace(go.Candlestick(x=plot_data.index,
                open=plot_data['open'],
                high=plot_data['high'],
                low=plot_data['low'],
                close=plot_data['close'],
                name='Price'))

    # Add SuperTrend traces
    fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data['supertrend'],
                             mode='lines', name='SuperTrend', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data['upperband'],
                             mode='lines', name='Upper Band', line=dict(color='green', dash='dash')))
    fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data['lowerband'],
                             mode='lines', name='Lower Band', line=dict(color='red', dash='dash')))

    # # Set title and increase plot size
    # fig.update_layout(
    #     title_text=f"Price Bars and SuperTrend Indicator (Last {num_bars if num_bars else 'All'} Bars)",
    #     width=1200,  # Increase width
    #     height=800   # Increase height
    # )

    # Update x-axis to show datetime properly
    fig.update_xaxes(
        type='date',
        tickformat='%Y-%m-%d %H:%M',
        tickangle=45,
        nticks=20,
        tickmode='auto'
    )

    # Add range slider and buttons
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type='date'
        )
    )

    # Update y-axis labels
    fig.update_yaxes(title_text="Price", secondary_y=False)

    # Show the plot
    fig.show()

# Example usage:
plot_price_and_supertrend(result)  # Plot all bars
# plot_price_and_supertrend(result, num_bars=100)  # Plot last 100 bars
